In [8]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from dataset import SimulationData
import math
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


In [9]:
dataset = SimulationData(stencilNum=1000,samplePerStencil=50)
#dataset.normalizeData()

In [10]:
max(dataset.y[:,2])

tensor(0.9834, dtype=torch.float64)

In [11]:
class MLP(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(MLP, self).__init__()
            self.input_size = input_size
            self.hidden_size1  = hidden_size1
            self.hidden_size2 = hidden_size2
            self.output_size = output_size
            self.linear1 = nn.Linear(self.input_size, self.hidden_size1)
            #self.batchNorm1 = nn.BatchNorm2d(self.hidden_size1)
            self.relu1 = nn.ReLU()
            self.linear2 = nn.Linear(self.hidden_size1, self.hidden_size2)
            #self.batchNorm2 = nn.BatchNorm2d(self.hidden_size2)
            self.relu2 = nn.ReLU()
            self.linear3 = nn.Linear(self.hidden_size2, self.output_size)
            #self.batchNorm2 = nn.BatchNorm2d(self.hidden_size2)
            #self.relu3 = nn.ReLU()
            
        def forward(self, x):
            hidden1 = self.linear1(x)
            relu1  =self.relu1(hidden1)
            hidden2 = self.linear2(relu1)
            relu2 = self.relu2(hidden2)
            output = self.linear3(relu2)
            return output

In [5]:
class WeightedSum(nn.Module):
    def __init__(self):
        super(WeightedSum, self).__init__()
    
    def forward(self, G, input_features):
        G_split_size = 4
        G_ = G[:, :G_split_size]
        L = torch.matmul(torch.transpose(G, 1,0), input_features)
        L_ = torch.matmul(torch.transpose(input_features, 1, 0), G_)
        D = torch.matmul(L, L_)
        return D
    

In [13]:
class MainNet(nn.Module):
    def __init__(self, input_size1, hidden_size1_1,hidden_size1_2, output_size1,
                 input_size2, hidden_size2_1, hidden_size2_2, output_size2):
        
        super(MainNet, self).__init__()
        self.embeddingNetwork = MLP(input_size1,hidden_size1_1, hidden_size1_2, output_size1)
        self.weightedSum = WeightedSum()
        self.flatten = nn.Flatten(1,-1)
        self.fittingNetwork = MLP(input_size2,hidden_size2_1, hidden_size2_2, output_size2)
        
        
   
    def forward(self, x):
        norm_data = torch.nn.functional.normalize(x)
        print(f'input values: {x}\n shape: {x.shape}')
        embedding_output = self.embeddingNetwork(norm_data)
        print(f'embedding output: {embedding_output}\n shape: {embedding_output.shape}')

        #weighted_output = self.weightedSum(embedding_output, x)
        print(f'weighted output: {weighted_output}\n shape: {weighted_output.shape}')
        
        weighted_output_flattened = torch.flatten(embedding_output)
        #print(f'weighted flattened: {weighted_output_flattened}\n shape: {weighted_output_flattened.shape}')
        
        fitting_output = self.fittingNetwork(weighted_output_flattened)
        #print(f'fitting output: {fitting_output}\n shape: {fitting_output.shape}')

        return fitting_output

In [14]:
input_size1 = dataset.x.shape[2]
hidden_size1_1 = 32
hidden_size1_2 = 64
output_size1 = 256

input_size2 = 256*50
hidden_size2_1 = 64
hidden_size2_2 = 32
output_size2 = 3

net = MainNet(input_size1, hidden_size1_1, hidden_size1_2, output_size1, input_size2,hidden_size2_1, hidden_size2_2, output_size2)

In [15]:
train, test = torch.utils.data.random_split(dataset, [len(dataset)-100,100])

In [16]:
len(train)

892

In [17]:
loader_train = DataLoader(dataset= dataset, batch_size = 1, num_workers=1)
loader_test = DataLoader(dataset= dataset, batch_size = 1, num_workers=1)

In [18]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

criterion = nn.MSELoss()

In [19]:
def test():
    net.eval()
    with torch.no_grad():
        losses = []
        counter = 0
        for i, data in enumerate(loader_test):
            counter +=1
            x_batch,y_batch = data
            for i in range(len(x_batch)):
                x = x_batch[i]
                y = y_batch[i]
                output1, output2, output3 = net(x.float())
                loss1 = criterion(output1.reshape(1).float(), y[[0]].float())
                loss2 = criterion(output2.reshape(1).float(), y[[1]].float())
                loss3 = criterion(output3.reshape(1).float(), y[[2]].float())
                loss = (loss1*10)+(loss2*10)+(loss3*10)
                losses.append(loss.item())
    print(f'avg testing loss: {sum(losses)/len(losses)}')
    return losses

In [20]:
def train_one_epoch():
    net.train()
    losses = []
    counter = 0
    eval_loss = []
    info = []
    for i, data in enumerate(loader_train):
        counter +=1
        x_batch,y_batch = data
        info.append({})
        for i in range(len(x_batch)):
            x = x_batch[i]
            y = y_batch[i]
            info[counter-1]["x"] = x
            info[counter-1]["y"] = y
            optimizer.zero_grad()
            output1, output2, output3 = net(x.float())
            loss1 = criterion(output1.reshape(1).float(), y[[0]].float())
            loss2 = criterion(output2.reshape(1).float(), y[[1]].float())
            loss3 = criterion(output3.reshape(1).float(), y[[2]].float())
            loss = (loss1*10)+(loss2*10)+(loss3*10)
            info[counter-1]["output"] = [output1.item(),output2.item(),output3.item()]
            info[counter-1]["loss"] = loss.item()
            loss.backward()
            optimizer.step()
            #print(x[0])
            #print(output1.item(), output2.item(), output3.item())
            #print(y[0].item(), y[1].item(), y[2].item())
            #print(loss.item())
            #print()
            #if(loss.item()>=0.20):
            losses.append(loss.item())

        if counter%10000 == 0:
           # print(losses[-1])
            print(output1.item(), output2.item(), output3.item())
            print(y[0].item(), y[1].item(), y[2].item())
            print(loss.item())
            print()
    print(f'avg train loss: {sum(losses)/len(losses)}')
    return losses,eval_loss, info

In [21]:
val_losses = []
train_losses = []
infos_train = []
infos_val = []

In [22]:
for i in tqdm(range(300)):
    val_losses.extend(test())
    train_lossTemp,val_lossTemp, info = train_one_epoch()
    train_losses.extend(train_lossTemp)
    infos_train.extend(info)
    #infos_val.extend(info)


  0%|                                                                                          | 0/300 [00:01<?, ?it/s]

input values: tensor([[ 3.6117e+00,  1.7368e+00,  0.0000e+00, -5.8388e-02,  8.9145e-05,
          2.5293e-01,  9.8683e-01,  1.2620e-01,  2.7105e+00,  3.1783e+00],
        [ 3.9521e+00,  1.6311e+00, -1.0477e-02, -6.8865e-02,  9.5746e-05,
          2.6421e-01,  9.6327e-01,  1.6927e-01,  3.0509e+00,  3.5187e+00],
        [ 3.2205e+00,  1.7368e+00,  8.9690e-03, -4.9419e-02,  8.9146e-05,
          2.4028e-01,  9.9870e-01,  1.1552e-01,  2.3193e+00,  2.7871e+00],
        [ 3.5705e+00,  2.0671e+00,  1.2356e-02, -4.6032e-02,  6.8669e-05,
          2.5769e-01,  1.0080e+00,  7.3164e-02,  2.6693e+00,  3.1371e+00],
        [ 3.6084e+00,  1.5941e+00, -4.9450e-03, -6.3333e-02,  9.8053e-05,
          2.4962e-01,  9.7121e-01,  1.6976e-01,  2.7072e+00,  3.1750e+00],
        [ 3.3752e+00,  2.0404e+00,  1.4825e-02, -4.3563e-02,  7.0314e-05,
          2.5124e-01,  1.0136e+00,  6.9353e-02,  2.4740e+00,  2.9418e+00],
        [ 3.7080e+00,  1.6671e+00, -4.5090e-03, -6.2897e-02,  9.3492e-05,
          2.5503e-

NameError: name 'weighted_output' is not defined

In [ ]:
#fig, ax = plt.subplots()
#for i, el in enumerate(infos_train):
    #x = dataset.original_x[i][-2]
    #y = dataset.original_x[i][-1]
    #print(x, y)
    #out = el["output"][2]
    #y = el["y"][2].item()
    #print(pos)
    #ax.scatter(x = x, y = y)


In [ ]:
def averageOut(list_nums, n):
    new_list = []
    temp_sum = 0
    temp_length = 0
    counter = 0
    average = 0
    temp_list = []
    final_list = []
    for i, num in enumerate(list_nums):
        counter+=1
        temp_list.append(num)
        if counter == n:
            avg = sum(temp_list) / len(temp_list)
            final_list.append(avg)
            temp_list = []
            counter = 0
    if len(temp_list) != 0:
        avg = sum(temp_list) / len(temp_list)
        final_list.append(avg)
    return final_list

In [ ]:
plt.yscale("log")
plt.plot(averageOut(train_losses, 1000), color="red", label="training loss")
plt.plot(averageOut(val_losses, 1000), color="blue", label="validation loss")
plt.title("validation & training loss (log scale)")
plt.xlabel("iteration")
plt.ylabel("loss")
plt.legend()
plt.savefig("./images/loss_4.png", dpi=1500)

In [ ]:
torch.save(net.state_dict(), "./models/net_C=1000_S=50_E=300.pth")


In [ ]:
net.load_state_dict(torch.load("./models/net.pth"))